In [1]:
from line_solver import *
import numpy as np
GlobalConstants.set_verbose(VerboseLevel.STD)

In [2]:
# Series of Fork-Join Networks
model = Network('model')

In [3]:
# Create series of fork-join stages
delay = Delay(model, 'Delay1')
queue1 = Queue(model,'Queue1',SchedStrategy.PS)
queue2 = Queue(model,'Queue2',SchedStrategy.PS)
fork = Fork(model,'Fork')
join = Join(model,'Join', fork)
queue3 = Queue(model,'Queue3',SchedStrategy.PS)
queue4 = Queue(model,'Queue4',SchedStrategy.PS)
fork2 = Fork(model,'Fork2')
join2 = Join(model,'Join2', fork2)

jobclass1 = ClosedClass(model, 'class1', 10, delay, 0)

In [4]:
# Service configurations
queue1.set_service(jobclass1, Exp(1.0))
queue2.set_service(jobclass1, Exp(1.0))
delay.set_service(jobclass1, Exp(0.5))
queue3.set_service(jobclass1, Exp(1.0))
queue4.set_service(jobclass1, Exp(1.0))

In [5]:
# Series fork-join routing
P = model.init_routing_matrix()

# First fork-join stage
P.set(jobclass1, jobclass1, delay, fork, 1.0)
P.set(jobclass1, jobclass1, fork, queue1, 1.0)
P.set(jobclass1, jobclass1, fork, queue2, 1.0)
P.set(jobclass1, jobclass1, queue1, join, 1.0)
P.set(jobclass1, jobclass1, queue2, join, 1.0)
# Between stages: join -> fork2
P.set(jobclass1, jobclass1, join, fork2, 1.0)
# Second fork-join stage
P.set(jobclass1, jobclass1, fork2, queue3, 1.0)
P.set(jobclass1, jobclass1, fork2, queue4, 1.0)
P.set(jobclass1, jobclass1, queue3, join2, 1.0)
P.set(jobclass1, jobclass1, queue4, join2, 1.0)
P.set(jobclass1, jobclass1, join2, delay, 1.0)

model.link(P)

In [6]:
# Solve with multiple methods
solvers = [JMT(model, seed=23000), MVA(model)]
for i, solver in enumerate(solvers):
    print(f'SOLVER {i+1}: {solver.get_name()}')
    avgTable = solver.avg_table()

SOLVER 1: JMT


Station JobClass   QLen   Util  RespT  ResidT   ArvR   Tput
 Delay1   class1 1.5890 1.5890 2.0056  2.0056 0.7887 0.7887
 Queue1   class1 2.7414 0.7973 3.4869  3.4869 0.7924 0.7924
 Queue2   class1 2.5448 0.7868 3.3743  3.3743 0.7948 0.7948
   Join   class1 2.8394 0.8597 1.9349  1.9349 0.7937 0.7937
 Queue3   class1 2.5811 0.7741 3.3346  3.3346 0.7933 0.7933
 Queue4   class1 2.7689 0.8166 3.5610  3.5610 0.7930 0.7930
  Join2   class1 3.1710 0.8682 2.0372  2.0372 0.7890 0.7890
SOLVER 2: MVA
Station JobClass   QLen   Util  RespT  ResidT  ArvR  Tput
 Delay1   class1 1.6941 1.6941 2.0000  2.0000 0.847 0.847
 Queue1   class1 5.5373 0.8470 6.5373  6.5373 0.847 0.847
 Queue2   class1 5.5373 0.8470 6.5373  6.5373 0.847 0.847
   Join   class1 5.5373 0.0000 3.2686  3.2686 0.847 0.847
 Queue3   class1 5.5373 0.8470 6.5373  6.5373 0.847 0.847
 Queue4   class1 5.5373 0.8470 6.5373  6.5373 0.847 0.847
  Join2   class1 5.5373 0.0000 3.2686  3.2686 0.847 0.847
